In [60]:
import pandas as pd
import json
import time

In [2]:
! pip install datasets transformers[sentencepiece]
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off
!pip install git+https://github.com/huggingface/peft
!pip install git+https://github.com/huggingface/accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-xle1d4dy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-xle1d4dy
  Resolved https://github.com/huggingface/peft to commit ce925d844a0bc54b951fcb69229dfe740c9afa45
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.8.2-py3-none-any.whl size=183371 sha256=cbaa54731222ebb1784b5853bff5881090f95e7e6759eaa2f4a60d84a7a66f7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-7pe76gmi/wheels/e1/0a/cc/243fa4389de86b6a8c8a6ac6511d2227d3e10934d3b19e5f5e
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0
    Uninstalling peft-0.4.0:
      Successfully uninstalled peft-0.4.0
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-q408r1e0
  Running command git clone --filter=blob:none 

In [1]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer
from contextlib import nullcontext
from transformers import (
    LlamaForCausalLM, 
    LlamaTokenizer, 
    TrainerCallback, 
    default_data_collator, 
    Trainer, 
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    TaskType,
    prepare_model_for_int8_training,
    PeftModel
)


/home/eyaya_eneyew/week_7/LLM_Finetuning_For_Amharic_Ad_Generation/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

In [3]:
def load_model(bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """
    model_name = "/model/Llama-2-7b-hf"
    #PT_DIR = "/model/llama-2-amharic-3784m"
    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{21960}MB'
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    # Load model
    model = LlamaForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        torch_dtype=torch.float16 #max_memory = {i: max_memory for i in range(n_gpus)},
    )
    #model = PeftModel.from_pretrained(model, PT_DIR)
    # Load model tokenizer with the user authentication token
    
    #tokenizer = LlamaTokenizer.from_pretrained(model_name)
    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

In [4]:
################################################################################
# transformers parameters
################################################################################

# The pre-trained model from the Hugging Face Hub to load and fine-tune
# model_name = "/model/Llama-2-7b-hf"

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

In [5]:
bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(bnb_config)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.06s/it]


In [6]:
# Load dataset
dataset_name = "/data/fine_tun_data2.json"
dataset = load_dataset("json", data_files = dataset_name, split = "train")

In [7]:
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Number of prompts: 1611
Column names are: ['output', 'instruction', 'input']


In [8]:
dataset[randrange(len(dataset))]

{'output': 'advertisement',
 'instruction': 'Categorize the input into one of the 2 categories:\n\nadvertisement\nnot advertisement\n\n',
 'input': 'ADVERTISMENT መልካም ነገር ከሉሲ የአጥንትና የመገጣጠሚያ ቀዶ ሕክምና ማእከል ለአጥንትና ለመገጣጠሚያ ችግሮችና ህመሞች አይነተኛ መፍትሄ የምንሰጣቸው የህክምና አገልግሎቶች ✔ማንኛውም ስብራትና ውልቃት ሕክምና ✔የጉልበትና ዳሌ መገጣጠሚያ ችግር ✔ውስብስብ ከባድ ስብራቶችና ውልቃቶች ✔የጡንቻና የጅማት ጉዳት ህመሞችን ማከም ✔የሕፃናት እና አዋቂዎች እጅና እግር መጣመም ማስተካከል ✔ስብራቶችንና ውልቃቶችን ያለ ቀዶ ሕክምና ማከም ✔በቀዶ ሕክምና የገቡ የተለያዩ ብረቶችን ማውጣት ✔የፕላስቲክ ሰርጀሪ ሕክምና ✔በዳሌና ዳሌ ገንዳ ስብራቶችን በፍሎሮስኮፒ የታገዘ ሕክምና ማድረግ 📌ልምድና ብቃት ባላቸው ስፔሻሊስት እና ሰብ ስፔሻሊስት ሀኪሞች እጅግ ዘመናዊ በሆኑ የህክምና መሳሪያወች በመታገዝ ሁሉንም የህክምና አገልግሎቶች በተመጣጣኝ ዋጋ ያገኛሉ‼ አድራሻ : አየር ጤና ጅማበር ፖሊስ ጣቢያ ጎን 📲 0913468103 0953912229'}

In [9]:
len(dataset)

1611

In [10]:
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample

In [11]:
create_prompt_formats(dataset[randrange(len(dataset))])

{'output': 'advertisement',
 'instruction': 'Categorize the input into one of the 2 categories:\n\nadvertisement\nnot advertisement\n\n',
 'input': 'ADVERTISMENT አፊያ ሙሀመድ ከፍተኛ የባህል ሕክምና እና ዘመናዊ የዋግምት አገልግሎት የምንሰጣቸው የባህል ህክምናዎች ➢ ለውጭና ለውስጥ ኪንታሮት ➢ ለማድያት ➢ ለሱኳር በሽታ ➢ለጉበት(ለወፍ በሽታ) ➢ለጨጎራ ህመም ➢ለስፈተወሲብ ➢ለደም ግፊት ➢ለአስም ወይም ሳይነስ ➢ለሚጥል በሺታ ➢ ለእሪህና ቁርጥማት ➢ለራስ ህመም (ማይግሪን) ➢ለቺፌ ና ለጭርት ➢ለቋቁቻና ፎረፎር ➢ለእጢና ለእባጭ ➢ለወገብ ህመም ➢ለመካንነት ለወድም ለሴትም ➢ለጆሮና ለአይን ህመም ➢ለሆድ ህመም ➢ዘመናዊ የዋግምት አገልግሎት በተቋማችን እንሰጣለን። 👉ከኢትዮጵያ ባህላዊ ህክምና አዋቂዎች ማህበር በዘርፉ ህጋዊ የባህል ህክምና ፍቃድ ያለን ነን። አድራሻ:አዲስ አበባ አየር ጤና ስልክ ቁጥር 📲0927506650 📲0987133734 📲0939605455 ቴሌግራም ቻናላችን ntvE5NmM0',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the input into one of the 2 categories:\n\nadvertisement\nnot advertisement\n\n\n\nInput:\nADVERTISMENT አፊያ ሙሀመድ ከፍተኛ የባህል ሕክምና እና ዘመናዊ የዋግምት አገልግሎት የምንሰጣቸው የባህል ህክምናዎች ➢ ለውጭና ለውስጥ ኪንታሮት ➢ ለማድያት ➢ ለሱኳር በሽታ ➢ለጉበት(ለወፍ በሽታ)

In [12]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [13]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [14]:
def preprocess_dataset(tokenizer, max_length, seed, dataset):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["instruction", "input", "output", "text"],
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [15]:
# Random seed
seed = 22

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

Found max lenth: 2048
Preprocessing dataset...


In [16]:
print(preprocessed_dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1611
})


In [17]:
print(tokenizer.decode(preprocessed_dataset[24]['input_ids']))

<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Categorize the input into one of the 2 categories:

advertisement
not advertisement



Input:
የማስታወቂያ ግብዣ👇 የተከበራችሁ ቤተሰቦች ከ53,700 በላይ ተከታይ ባለው እና በደቂቃዎች ውስጥ በሺህ የሚቆጠሩ ጎብኝ ባለው በዚህ የቴሌግራም ቻናል ላይ በልዩ ቅናሽ ምርትና አገልግሎታችሁን እንድታስተዋውቁ በአክብሮት እጋብዛለሁ‼ ✔Youtube ቻናል … ✔የትምህርት ተቋማት… ✔የህክምና ማእከላት (ባህላዊ /ዘመናዊ)… ✔የትራንስፖርት አገልግሎት… ✔የስራ አገናኝ ኤጀንሲዎች…… ✔ሆቴል፣ካፌና ሬስቶራንቶች ✔ሌሎች የምትሰጧቸውንም አገልግሎቶችን ማስተዋወቅ ለምትፈልጉ ኑ አብረን እንስራ ብያለሁ‼ ማስታወቂያዎን በዚህ ይላኩ👇

### Response:
advertisement

### End


In [18]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

In [19]:
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [20]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [21]:
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          #warmup_ratio,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """
    try:
        # Enable gradient checkpointing to reduce memory usage during fine-tuning
        
        model.train()
        embedding_size = model.get_input_embeddings().weight.shape[0]

        if len(tokenizer) != embedding_size:
            print("resize the embedding size by the size of the tokenizer")
            model.resize_token_embeddings(len(tokenizer))
            
        model.resize_token_embeddings(len(tokenizer))

        print('loading the pretrained model from config')
        # Prepare the model for training
        model = prepare_model_for_kbit_training(model)
        model = PeftModel.from_pretrained(model, "/model/llama-2-amharic-3784m")
        # Get LoRA module names
        target_modules = find_all_linear_names(model)

        # Create PEFT configuration for these modules and wrap the model to PEFT
        peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
        model = get_peft_model(model, peft_config)

        # Print information about the percentage of trainable parameters
        print_trainable_parameters(model)

        # Training parameters
        trainer = Trainer(
            model = model,
            train_dataset = dataset,
            args = TrainingArguments(
                per_device_train_batch_size = per_device_train_batch_size,
                gradient_accumulation_steps = gradient_accumulation_steps,
                warmup_steps = warmup_steps,
                max_steps = max_steps,
                learning_rate = learning_rate,
                fp16 = fp16,
                logging_steps = logging_steps,
                output_dir = output_dir,
                optim = optim,
            ),
            data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
        )

        model.config.use_cache = False

        do_train = True

        # Launch training and log metrics
        print("Training...")

        if do_train:
            train_result = trainer.train()
            metrics = train_result.metrics
            trainer.log_metrics("train", metrics)
            trainer.save_metrics("train", metrics)
            trainer.save_state()
            print(metrics)

        # Save model
        print("Saving last checkpoint of the model...")
        os.makedirs(output_dir, exist_ok = True)
        trainer.model.save_pretrained(output_dir)

        # Free memory for merging weights
    except Exception as err:
        print("Error:",err)
        del model
        del trainer
        torch.cuda.empty_cache()

In [43]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.05

# Bias
bias = "none"

# Task type
task_type = "CAUSAL_LM"

In [50]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Initial learning rate (AdamW optimizer)
learning_rate = 1e-5

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 100

# Linear warmup steps from 0 to learning_rate
warmup_steps = 20

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
logging_steps = 1

In [51]:
fine_tune(model,
      tokenizer,
      preprocessed_dataset,
      lora_r,
      lora_alpha,
      lora_dropout,
      bias,
      task_type,
      per_device_train_batch_size,
      gradient_accumulation_steps,
      warmup_steps,
      max_steps,
      learning_rate,
      fp16,
      logging_steps,
      output_dir,
      optim)


loading the pretrained model from config


LoRA module names: ['base_layer']
All Parameters: 4,453,765,120 || Trainable Parameters: 359,792,640 || Trainable Parameters %: 8.078392782419563
Training...


Step,Training Loss
1,8.400400
2,7.342400
3,13.039400
4,8.154800
5,8.285000
6,6.589800
7,6.836900
8,8.888700
9,6.511000
10,7.240900


***** train metrics *****
  epoch                    =       0.06
  total_flos               =   532552GF
  train_loss               =     3.0567
  train_runtime            = 0:03:46.86
  train_samples_per_second =      0.441
  train_steps_per_second   =      0.441
{'train_runtime': 226.8638, 'train_samples_per_second': 0.441, 'train_steps_per_second': 0.441, 'total_flos': 571823530131456.0, 'train_loss': 3.056667756438255, 'epoch': 0.06}
Saving last checkpoint of the model...


In [80]:
def create_prompt_formats_for_test(sample):
    'Categorize the input into one of the 2 categories:\n\nadvertisement\nnot advertisement\n\n'
    """
    Format various fields of the sample ('input', 'label',)
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\nCategorize the input into one of the two categories output only either advertisement or not advertisement. NOTHING ELSE.\n\n"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context,response,end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    #sample["input"] = formatted_prompt

    return sample

In [53]:
dataset[10]

{'output': 'advertisement',
 'instruction': 'Categorize the input into one of the 2 categories:\n\nadvertisement\nnot advertisement\n\n',
 'input': 'የምስራች! 🇪🇹 በሀገራችን ምርት እንጠቀም! እንኩራ! የRingAround App ይሞክሩን tikvah downloadringaroundapp በውጭ ሀገር ለምትኖሩ ኢትዮጵያውያን! ሳያመልጣችሁ በሀገራችን የቴክኖሎጂ ጠቢባን ተዘጋጅቶ የቀረበውን አለም አቀፉን የRingAround Appን በስልካችሁ ላይ ቶሎ በመጫን ወደ ሀገር ቤት ይደውሉ!'}

In [54]:
sample = dataset[10]

prompt = create_prompt_formats_for_test(sample)

## Inference using Instruction or Question Only

In [55]:
input_text = f"Instruction: {prompt['text']}"

In [56]:
input_text

'Instruction: Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the input into one of the 2 categories output only either advertisement or not advertisement. NOTHING ELSE.\n\n\n\nInput:\nየምስራች! 🇪🇹 በሀገራችን ምርት እንጠቀም! እንኩራ! የRingAround App ይሞክሩን tikvah downloadringaroundapp በውጭ ሀገር ለምትኖሩ ኢትዮጵያውያን! ሳያመልጣችሁ በሀገራችን የቴክኖሎጂ ጠቢባን ተዘጋጅቶ የቀረበውን አለም አቀፉን የRingAround Appን በስልካችሁ ላይ ቶሎ በመጫን ወደ ሀገር ቤት ይደውሉ!\n\n### Response:'

In [63]:
#text_i = 'ADVERTISMENT 🏑 መልካም ገና 🏑 🔜 💥ለ3 ቀናት ብቻ የሚቆይ ታላቅ'
text_i = 'ማስታወቂያ! ድሮጋ የፊዚዮቴራፒ ህክምና ልዩ ክሊኒክ፦ ዘመኑ ባፈራቸው የመጨረሻ የህክምና ቴክኖሎጂ በመታገዝና በሙያው ስፔሻላይዝ ባደረጉ የፊዝዮቴራፒ ሃኪሞች በህፃናት ላይ በሚከሰቱ የጭንቅላት ላይ ጉዳቶች አልያም ማንኛውንም የእንቅስቃሴ ጉድለትና የአካል አለመታዘዝ ችግሮች ፍቱን የሆነ ህክምና በማድረግ ወደ ተሟላ የሰውነት እንቅስቃሴ እንመልሳለን። አአ ሚግሬሸን ጀርባ ለበለጠ መረጃ 0974 95 95 95 like us on FB @drogaphysiotherapy'

In [31]:
text_i = 'ታላቅ ቅናሽ ከዲ ኤም ሲ (DMC) ሪል እስቴት 5% ከፈለው ይመዝገቡ የሚፈልጉትን ቤት ይምረጡ ከ ስቲድዮ እስከ ባለ 4 መኝታ ቤቶች በተለያየ የካሬ ኣማራጭ ዘመናዊ አፓርትመንቶችን በመሸጥ ላይ ነን'

In [82]:
query = f'Instruction: Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the input into one of the 2 categories output only either advertisement or not advertisement. NOTHING ELSE.\n\n\n\nInput:\n{text_3}\n\n### Response:'

In [33]:
query = 'what medicine should I take if I got a flu'

In [81]:
text_3 = 'ደና ዋላቹ'

In [84]:
# Tokenize the input
input_ids = tokenizer.encode(query, return_tensors="pt").to(model.device)

# Measure inference time
start_time = time.time()
batch = tokenizer(query, return_tensors="pt")
batch = {k: v.to("cuda") for k, v in batch.items()}
#start = time.perf_counter()
# Generate predictions
#output = model.generate(input_ids, max_length=500, temperature=1.0, top_k=50, top_p=0.95, num_return_sequences=1)
output = model.generate(**batch,  max_new_tokens=400,
                do_sample=True,
                top_p=1,
                temperature=1,
                use_cache=True,
                top_k=1,
                repetition_penalty=1,
                length_penalty=1)
generated_output = tokenizer.decode(output[0], skip_special_tokens=True)

end_time = time.time()

# Calculate and print the inference time
inference_time = end_time - start_time


In [86]:
print(generated_output)

Instruction: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Categorize the input into one of the 2 categories output only either advertisement or not advertisement. NOTHING ELSE.



Input:
ደና ዋላቹ

### Response:
advertisement

### End###
### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### End###
### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### End###
### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### End###
### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### End###
### Instruction:
Below

In [277]:
# Print the formatted input
print(f"======")
print(f"Input:\n======\n{input_text}\n")
print(f"======================")
print(f"Generated Output:\n======================\n{generated_output}\n")
print(f"=========================================")
print(f"Inference Time:{inference_time} seconds\n==========================================")

Input:
Instruction: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
ጥቆማ📌የዩኒቨርሲቲ ተማሪዎች የጥሪ ቀናችሁን ከተቋማችሁ ትክክለኛ የፌስቡክ ገፅ ወይም ከድህረገፅ ላይ ካላገኛችሁ እንዲሁም በማህተም የተደገፉ የማስታወቂያ ፅሁፎችን ካላያችሁ ልታምኑ አይገባም። በመሆኑንም የተጠራችሁበትን ቀን ለማወቅ ቀጥታ በዩኒቨርሲታያችሁ የፌስቡክ ገፅ እየገባችሁ አረጋግጡ። እንዲሁም የተማሪ ህብረት ተወካይ ስልክ እና የሬጅስትራር ቢሮ ስልክ ቁጥሮችን መያዝ ባህላችሁ አድርጉ። @tsegabwolde @tikvahethiopia

Generated Output:
Instruction: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
ጥቆማ📌የዩኒቨርሲቲ ተማሪዎች የጥሪ ቀናችሁን ከተቋማችሁ ትክክለኛ የፌስቡክ ገፅ ወይም ከድህረገፅ ላይ ካላገኛችሁ እንዲሁም በማህተም የተደገፉ የማስታወቂያ ፅሁፎችን ካላያችሁ ልታምኑ አይገባም። በመሆኑንም የተጠራችሁበትን ቀን ለማወቅ ቀጥታ በዩኒቨርሲታያችሁ የፌስቡክ ገፅ እየገባችሁ አረጋግጡ። እንዲሁም የተማሪ ህብረት ተወካይ ስልክ እና የሬጅስትራር ቢሮ ስልክ ቁጥሮችን መያዝ ባህላችሁ አድርጉ። @tsegabwolde @tikvahethiopia

### Response:
ተማሪዎችን ለጥሪ ቀናችሁን ከተቋማችሁ ትክክለኛ የፌስቡክ ገፅ ወይም ከድህረገፅ ላይ ካላቀዘቁ እንዲሁም በማህተም የተደገፉ የማስታወቂያ ፅሁፎችን ካላያችሁ ልታምኑን አይደለም። በዚህምም ተጠርጣችሁን የመወጣች